In [1]:
from functools import partial

import torch

from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import (
    AutoModel,
    AutoTokenizer,
    DataCollatorWithPadding,
    get_constant_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
)

from data import ContrastivePairDS, pair_collate, load_kasa_regression, standardize_df, tokenize
from model import DeltaModel
from collections import defaultdict
from tqdm.auto import tqdm

import wandb
wandb.init(project="delta",name="init_v7")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rahul-e-dev. Use `wandb login --relogin` to force relogin


In [2]:
def train_step(model, batch, optimizer, scheduler):
    optimizer.zero_grad()
    model.train()
    device = next(model.parameters()).device

    batch = {k: v.to(device) for k,v in batch.items()}
    losses = model(batch)
    losses['loss'].backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    return losses

@torch.no_grad()
def val_step(model, batch):
    model.eval()
    device = next(model.parameters()).device
    batch = {k: v.to(device) for k,v in batch.items()}
    losses = model(batch)
    return losses

In [3]:
ds = load_kasa_regression()
tokenizer = AutoTokenizer.from_pretrained('DeepChem/ChemBERTa-77M-MTR')
tok_func = partial(tokenize, tokenizer=tokenizer)
ds = ds.map(tok_func, num_proc=8).remove_columns(['smiles'])

tokenizer.pad_token = tokenizer.eos_token
padding_collator = DataCollatorWithPadding(tokenizer)

c:\Users\rahul\mambaforge\envs\bio\lib\site-packages\astartes\samplers\extrapolation\scaffold.py:44: NoMatchingScaffold: No matching scaffold was found for the 3 molecules corresponding to indices {23234, 36709, 11879}
  warnings.warn(


Map (num_proc=8):   0%|          | 0/39777 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/4972 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/4972 [00:00<?, ? examples/s]

In [4]:
@torch.no_grad()
def embed_split(ds_split, model):
    embeddings = []
    device = next(model.parameters()).device
    for batch in DataLoader(ds_split, shuffle=False, batch_size=1024, collate_fn=padding_collator):
        batch = {k:v.to(device) for k,v in batch.items()}
        batch_embeddings = model.embed_batch(batch)
        embeddings.append(batch_embeddings.cpu())

    embeddings = torch.cat(embeddings, dim=0)
    return embeddings.to(device)

@torch.no_grad()
def get_actual_diffs_for_split(ds_split):
    val_scores = torch.tensor(ds_split['score'])
    actual_diffs = val_scores.unsqueeze(-1) - val_scores.unsqueeze(0)
    return actual_diffs.cpu().numpy()


@torch.no_grad()
def get_predicted_diffs_for_split(ds_split, model):
    embeddings = embed_split(ds_split, model)

    N = embeddings.shape[0]
    pred_delta = torch.zeros((N, N))
    for row_idx in tqdm(range(N)):
        pred_delta_row = model.get_delta(
            from_embedding=embeddings[[row_idx]].expand_as(embeddings),
            to_embedding=embeddings
        )
        pred_delta[row_idx, :] = pred_delta_row.squeeze()

    return pred_delta

In [5]:
def contrastive_pair_collate(batch, base_collator):
    batch_from = [item["from"] for item in batch]
    batch_to = [item["to"] for item in batch]

    batch_from = base_collator(batch_from)
    batch_to = base_collator(batch_to)

    return {"from": batch_from, "to": batch_to}


def update_dataloaders(train_split, val_split, model):
    train_ds = ContrastivePairDS(
        train_split,
        get_actual_diffs_for_split(train_split),
        get_predicted_diffs_for_split(train_split, model)
    )

    val_ds = ContrastivePairDS(
        val_split,
        get_actual_diffs_for_split(val_split),
        get_predicted_diffs_for_split(val_split, model)
    )

    train_dl = DataLoader(
        train_ds, 
        batch_size=512, 
        shuffle=True, 
        collate_fn=partial(pair_collate, base_collator=padding_collator),
        # num_workers=4
    )

    val_dl = DataLoader(
        val_ds, 
        batch_size=512, 
        shuffle=False, 
        collate_fn=partial(pair_collate, base_collator=padding_collator),
        # num_workers=4
    )

    return train_dl, val_dl

In [6]:
model = DeltaModel(AutoModel.from_pretrained('DeepChem/ChemBERTa-77M-MTR')).cuda()
train_dl, val_dl = update_dataloaders(ds['val'], ds['test'], model)

optimizer = AdamW(model.parameters(), lr=2e-4)
n_epochs = 40
n_steps = len(train_dl) * n_epochs
scheduler = get_cosine_schedule_with_warmup(optimizer, n_steps*0.01, n_steps*(1-0.01))
# scheduler = get_constant_schedule_with_warmup(optimizer, n_steps*0.01)

wandb.watch(model, log='gradients', log_freq=10)

Some weights of the model checkpoint at DeepChem/ChemBERTa-77M-MTR were not used when initializing RobertaModel: ['regression.dense.bias', 'norm_mean', 'regression.dense.weight', 'norm_std', 'regression.out_proj.bias', 'regression.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions an

  0%|          | 0/4972 [00:00<?, ?it/s]

  0%|          | 0/4972 [00:00<?, ?it/s]

[]

In [7]:
def step_epoch(e, model, train_dl, val_dl, optimizer, scheduler):
    train_loss_accumulator = defaultdict(float)
    for batch in tqdm(train_dl, total=len(train_dl)):
        train_losses = train_step(model, batch, optimizer, scheduler)
        for k,v in train_losses.items():
            train_loss_accumulator[k] += v.item()

    train_loss_accumulator = {
        f'train/{k}': round(v/len(train_dl), 4) 
        for k,v in train_loss_accumulator.items()
    }

    wandb.log(dict(train_loss_accumulator) | {'train/epoch': e})


    if e%1 == 0:
        val_loss_accumulator = defaultdict(float)
        for batch in val_dl:
            val_losses = val_step(model, batch)
            for k,v in val_losses.items():
                val_loss_accumulator[k] += v.item()
            
        val_loss_accumulator = {
            f'val/{k}': round(v/len(val_dl), 4) 
            for k,v in val_loss_accumulator.items()
        }

        wandb.log(dict(val_loss_accumulator) | {'val/epoch': e})

In [8]:
for e in tqdm(range(n_epochs+1)):
    step_epoch(e, model, train_dl, val_dl, optimizer, scheduler)

    if e%1 == 0:
        train_dl, val_dl = update_dataloaders(ds['val'], ds['test'], model)

  0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4972 [00:00<?, ?it/s]

  0%|          | 0/4972 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/4972 [00:00<?, ?it/s]

  0%|          | 0/4972 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4972 [00:00<?, ?it/s]

  0%|          | 0/4972 [00:00<?, ?it/s]

In [9]:
wandb.finish()

train/delta_loss,██▅▃▆▃▂▄▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train/id_loss,▁▁▂▂▂▇▃▃▂▂▁▁▁▁▂▂▂▂▆█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/inv_loss,▁▁▅█▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,██▅▄▆▄▃▄▃▂▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/delta_loss,▂▂▂▁▁█▃▂▅▂▁▁▁▁▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
val/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
val/id_loss,▁▂█▁▂▄▂▆▅▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/inv_loss,▁▁█▁▁▅▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,▂▂▄▁▁█▃▃▆▂▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
train/delta_loss,5.6321
